## IMPORTING LIBRARIES

In [1]:
import pandas as pd
from selenium import webdriver
import time
import datetime
#
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
#
import bs4
import urllib
#
import random

## SCRAPPING RANKING DATA

In [ ]:
data_old = pd.read_csv('bwf_historic_rankings.csv')
data_old = data_old.drop('Unnamed: 0', axis = 1)

In [ ]:
data_old.head()

In [ ]:
last_fetched_date = data_old['date'].max()
last_fetched_date = '2022-08-15'

In [ ]:
last_fetched_date

In [ ]:
driver = webdriver.Chrome()

driver.get('https://badmintonstatistics.net/Rankings')
driver.maximize_window()

In [ ]:
select = Select(driver.find_element(By.XPATH, '//*[@id="rankingPeriodsSelect"]'))
# select by visible text
select.select_by_index('0')

In [ ]:
#getting last ID vs historic data
last_current_week = driver.find_element(By.XPATH, '//*[@id="rankingPeriodsSelect"]/option[1]').text.replace('/', '-')
last_current_week

In [ ]:
dic_ranking = {'date':[], 'draw':[], 'top_100':[]}

In [ ]:
last_fetched_date

In [ ]:
date_listing = driver.find_element(By.XPATH, '//*[@id="rankingPeriodsSelect"]').text
date_listing = date_listing.replace(' ','').replace('/','-').split('\n')
date_listing = [x[6:10] + '-' + x[3:5] + '-' + x[0:2] for x in date_listing]
date_listing[0:5]

In [ ]:
date_listing_filtered = [x for x in date_listing if x > last_fetched_date]
display(date_listing_filtered[:5])
display(date_listing_filtered[-5:])

In [ ]:
for date in date_listing_filtered:

    for draw in ['MS','WS','MD','WD','XD']:
        
        driver.get('https://badmintonstatistics.net/Rankings?date=' + date + '&category=' + draw + '&country=%&page=1&pagesize=100')

        top_100 = driver.find_element(By.XPATH, '//*[@id="RankingList"]/div[2]/table/tbody')

        dic_ranking['date'].append(date)
        dic_ranking['draw'].append(draw)
        dic_ranking['top_100'].append(top_100.text)

        #time.sleep(random.randint(3,5))

In [ ]:
driver.get('https://badmintonstatistics.net/Rankings?date=' + date + '&category=' + draw + '&country=%&page=1&pagesize=100')

In [ ]:
driver.find_element(By.XPATH, '//*[@id="RankingList"]/div[2]/table/tbody/tr[1]/td[2]/img[2]').text

In [ ]:
driver.find_element(By.XPATH, '//*[@id="RankingList"]/div[2]/table/tbody').item

In [ ]:
dic_ranking['top_100'].append(top_100.text)

In [ ]:
delta = pd.DataFrame.from_dict(dic_ranking)
delta

In [ ]:
delta.to_csv('export_delta_top100_bwf.csv')

## CLEANING DATA

In [ ]:
def prep_data(df):

    df = pd.DataFrame(df.top_100.str.split('\n').tolist(), index=df[['date','draw']]).stack().reset_index()
    
    df.columns = ['date / draw', 'to_drop', 'content']

    df['date'] = df['date / draw'].str[0]

    df['draw'] = df['date / draw'].str[1]

    df['rank'] = df['content'].str.split(' ', 1).str[0]

    df['points'] = df['content'].str.split(' ').str[-1]

    df['country'] = df['content'].str.split(' ').str[-3].str.upper()
    
    df['name'] = df['content'].str.split(' ').str[1:-3]

    df['country'] = df['country'].str.replace('KONG','HONG KONG')
    df['country'] = df['country'].str.replace('TAIPEI','CHINESE TAIPEI')
    
    df['name'] = df['name'].apply(lambda x: ','.join(map(str, x)))
    df['name'] = df['name'].str.replace(',',' ').str.upper()
    df['name'] = df['name'].str.replace('CHINESE','')
    
    df['points'] = df['points'].astype('int')
    
    df['rank'] = df['rank'].astype('int')

    df = df.drop(['date / draw','content', 'to_drop'], axis = 1)
    
    display(df.info())
    display(df.head())
    
    return df

In [ ]:
pd.DataFrame(delta.top_100.str.split('\n').tolist(), index=delta[['date','draw']]).stack().reset_index()

In [ ]:
delta_prep = prep_data(delta)

In [ ]:
print(delta_prep.shape)
print(data_old.shape)

In [ ]:
data_full = pd.concat([delta_prep, data_old], ignore_index=True)

In [ ]:
data_full.head()

In [ ]:
data_full.to_csv('bwf_historic_rankings.csv')

## SCRAPPING TOURNAMENTS DATA

In [2]:
driver = webdriver.Chrome()

driver.get('https://bwfworldtour.bwfbadminton.com/calendar/?cyear=2018')
driver.maximize_window()

In [3]:
driver.find_element(By.XPATH, '//*[@id="page"]/div/div[1]/div/div/div/div').text.split('\n')[:10]

['JANUARY',
 'PRINCESS SIRIVANNAVARI Thailand Masters 2018',
 '09 - 14 JANUARY',
 'PRIZE MONEY USD $150,000',
 'Bangkok',
 'Thailand',
 'PERODUA Malaysia Masters 2018',
 '16 - 21 JANUARY',
 'PRIZE MONEY USD $350,000',
 'Bukit Jalil']

In [5]:
# identify elements with tagname <a>
lnks=driver.find_elements(By.TAG_NAME, "a")
# traverse list
for lnk in lnks:
   # get_attribute() to get all href
   print(lnk.get_attribute(href))

In [ ]:
driver.find_element(By.XPATH, '/html/body/div[2]/div/div[1]/div/div/div/div').href

In [ ]:
elems = driver.find_elements(By.XPATH, "/html/body/div[2]/div/div[1]/div/div/div/div")
for elem in elems:
    print(elem.get_attribute("href"))